## Modelling MBS prepayments with Python  

In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
Seasonality = {
    'January':  0.89,
    'February': 0.90,
    'March':    0.95,
    'April':    0.96,
    'May':      1.06,
    'June':     1.09,
    'July':     1.10,
    'August':   1.15,
    'September':1.13,
    'October':  1.01,
    'November': 0.86,
    'December': 0.90
}

def Seasonality_f(dizionario, Chiave):
    for key, value in dizionario.items():
        if key == Chiave:
            return value

In [3]:
AgeLoanSeason = {
-0.05: 48,
-0.04: 48,
-0.03: 48,
-0.02: 48,
-0.01: 48,
0.00:  30,
0.01:  12,
0.02:  12,
0.03:   6,
0.04:   6,
0.05:   6
}

def LoanSeason_f(dizionario, Chiave, Payment):
    ma = 0.05
    mi = -0.05
    Chiave = round(Chiave, 3)    
    Chiave = min(max(Chiave, mi),ma)

    for key, value in dizionario.items():
        val = value
        #print(key , "-", val)
        if round(key, 3) >= Chiave:
            return min(Payment/value,1.0)

In [4]:
Burnout = {
    0.0: 0.4,
    0.1: 0.45,
    0.2: 0.5,
    0.3: 0.6,
    0.4: 0.9,
    0.5: 0.95,
    0.6: 0.96,
    0.7: 0.97,
    0.8: 0.98,
    0.9: 0.99,
    1.0: 1}

def Burnout_f(dizionario, Chiave):    
    ma = 1.0
    mi = 0.0
    Chiave = round(Chiave, 1)    
    Chiave = min(max(Chiave, mi),ma)

    for key, value in dizionario.items():
        val = value
        #print(key , "-", val)
        if round(key, 3) >= Chiave:
            return value

### Which prepayment base factor model do you want to run?

In [5]:
BaseFactor = {
    -0.05: 5,
    -0.04: 6,
    -0.03: 7,
    -0.02: 8,
    -0.01: 9,
    0:    10,
    0.01: 20,
    0.02: 35,
    0.03: 45,
    0.04: 55,
    0.05: 65}

def BaseFactor_f(dizionario, Chiave):   
    ma = 0.05
    mi = -0.05
    Chiave = round(Chiave, 3)    
    Chiave = min(max(Chiave, mi),ma)
    
    for key, value in dizionario.items():
        val = value
        #print(key , "-", val)
        if round(key, 3) >= Chiave:
            return (1-(1-value/100)**(1/12))

In [6]:
from math import atan

In [7]:
def BaseFactor_S_Shape(Chiave):   
    ma = 0.05
    mi = -0.05
    Chiave = round(Chiave, 3)    
    Chiave = min(max(Chiave, mi),ma)
    
    result = (0.3-0.16*atan(123.11*(0.02-Chiave)))
    return (1-(1-result)**(1/12))

In [8]:
interest      = 0.06
years         = 30
payments_year = 12
mortgage      = 10000000
start_date    = (date(2016, 1, 1))
MarketYield   = 0.08
YieldDiff     = interest - MarketYield

In [9]:
rng = pd.date_range(start_date, periods = years*payments_year, freq = 'MS')
rng

DatetimeIndex(['2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
               '2016-09-01', '2016-10-01',
               ...
               '2045-03-01', '2045-04-01', '2045-05-01', '2045-06-01',
               '2045-07-01', '2045-08-01', '2045-09-01', '2045-10-01',
               '2045-11-01', '2045-12-01'],
              dtype='datetime64[ns]', length=360, freq='MS')

In [10]:
rng.name = "Payment Date"
rng

DatetimeIndex(['2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
               '2016-09-01', '2016-10-01',
               ...
               '2045-03-01', '2045-04-01', '2045-05-01', '2045-06-01',
               '2045-07-01', '2045-08-01', '2045-09-01', '2045-10-01',
               '2045-11-01', '2045-12-01'],
              dtype='datetime64[ns]', name='Payment Date', length=360, freq='MS')

In [11]:
df = pd.DataFrame(index = rng, 
                  columns=['Beg Bal','Payment','Interest', 'Principal', 'Prepayment','End Bal',
                          'BaseFactor', 'Burnout', 'Seasoning', 'Seasonality', 'SMM'], 
                  dtype = 'float')

In [12]:
df

,Beg Bal,Payment,Interest,Principal,Prepayment,End Bal,BaseFactor,Burnout,Seasoning,Seasonality,SMM
Payment Date,,,,,,,,,,,
2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2045-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2045-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2045-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.reset_index(inplace = True)
df

,Payment Date,Beg Bal,Payment,Interest,Principal,Prepayment,End Bal,BaseFactor,Burnout,Seasoning,Seasonality,SMM
0,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
355,2045-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,2045-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,2045-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,2045-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.index +=1
df

,Payment Date,Beg Bal,Payment,Interest,Principal,Prepayment,End Bal,BaseFactor,Burnout,Seasoning,Seasonality,SMM
1,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2016-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
356,2045-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,2045-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,2045-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,2045-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df.index.name = 'Period'
df

,Payment Date,Beg Bal,Payment,Interest,Principal,Prepayment,End Bal,BaseFactor,Burnout,Seasoning,Seasonality,SMM
Period,,,,,,,,,,,,
1,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2016-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
356,2045-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,2045-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,2045-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Choose the prepayment model you want to run

In [16]:
# Choose now the prepayment base factor model that you want to use
# If you type Model = 1, then the model will use the simple mapping,
# If you type Model = 2, then the model will use the s-shape model

# Model = 1
Model = 2

In [17]:
for period in range(1, len(df) + 1):
    if (period == 1):
        df.loc[period, 'Beg Bal'] = mortgage
    else:
        df.loc[period, 'Beg Bal'] = df.loc[period - 1, 'End Bal']
 
    BegBal = df.loc[period, 'Beg Bal'] 
    
    payment = -1 * np.pmt(interest/payments_year,     (years * payments_year) + 1 - period, BegBal)
    inte    = -1 * np.ipmt(interest/payments_year, 1, (years * payments_year) + 1 - period, BegBal)
    princ   = -1 * np.ppmt(interest/payments_year, 1, (years * payments_year) + 1 - period, BegBal)
    
    df.loc[period, ['BaseFactor', 'Burnout', 'Seasoning', 'Seasonality']] = 0
    
    # Seasonality
    Seas = Seasonality_f(Seasonality, df.loc[period, 'Payment Date'].month_name())        
    df.loc[period, 'Seasonality'] = Seas
    # Seasoning
    LoanS = LoanSeason_f(AgeLoanSeason, YieldDiff, period)
    df.loc[period, 'Seasoning'] = LoanS    
    # Burnout
    Burn = Burnout_f(Burnout, BegBal / mortgage)
    df.loc[period, 'Burnout'] = Burn    
    
    
    # *******************************************************************
    # Base Factor
    if Model == 1:
        BaseF = BaseFactor_f(BaseFactor, YieldDiff)
    else:
        BaseF = BaseFactor_S_Shape(YieldDiff)
    
    df.loc[period, 'BaseFactor'] = BaseF
    # *******************************************************************
    
    
    
    # SMM
    df.loc[period, 'SMM'] = \
        df.loc[period,'BaseFactor'] * df.loc[period, 'Burnout'] * df.loc[period, 'Seasoning'] * df.loc[period,'Seasonality']

    # Loan Repayment
    df.loc[period, 'Payment']    = payment
    df.loc[period, 'Interest']   = inte    
    df.loc[period, 'Principal']  = princ
    
    # Loan prepayment
    prep                         = df.loc[period, 'SMM'] * max(BegBal - princ,0)
    df.loc[period, 'Prepayment'] = prep
    
    df.loc[period, 'End Bal'] = max(BegBal - princ - prep,0)
    

In [18]:
df.head(10)

,Payment Date,Beg Bal,Payment,Interest,Principal,Prepayment,End Bal,BaseFactor,Burnout,Seasoning,Seasonality,SMM
Period,,,,,,,,,,,,
1,2016-01-01,1.000000e+07,59955.052515,50000.000000,9955.052515,1294.751726,9.988750e+06,0.00699,1.0,0.020833,0.89,0.000130
2,2016-02-01,9.988750e+06,59947.282089,49943.750979,10003.531110,2615.637482,9.976131e+06,0.00699,1.0,0.041667,0.90,0.000262
3,2016-03-01,9.976131e+06,59931.568657,49880.655136,10050.913521,4136.169082,9.961944e+06,0.00699,1.0,0.062500,0.95,0.000415
4,2016-04-01,9.961944e+06,59906.695578,49809.719723,10096.975855,5564.984556,9.946282e+06,0.00699,1.0,0.083333,0.96,0.000559
5,2016-05-01,9.946282e+06,59873.196285,49731.409921,10141.786364,7668.715570,9.928471e+06,0.00699,1.0,0.104167,1.06,0.000772
6,2016-06-01,9.928471e+06,59826.986136,49642.357411,10184.628725,9445.902595,9.908841e+06,0.00699,1.0,0.125000,1.09,0.000952
7,2016-07-01,9.908841e+06,59770.008566,49544.204754,10225.803812,11099.264847,9.887516e+06,0.00699,1.0,0.145833,1.10,0.001121
8,2016-08-01,9.887516e+06,59702.988772,49437.579411,10265.409361,13232.836429,9.864018e+06,0.00699,1.0,0.166667,1.15,0.001340
9,2016-09-01,9.864018e+06,59623.002961,49320.088182,10302.914779,14593.181620,9.839122e+06,0.00699,1.0,0.187500,1.13,0.001481


In [19]:
df.tail()

,Payment Date,Beg Bal,Payment,Interest,Principal,Prepayment,End Bal,BaseFactor,Burnout,Seasoning,Seasonality,SMM
Period,,,,,,,,,,,,
356,2045-08-01,63759.977756,12943.911489,318.799889,12625.111600,164.416441,50970.449715,0.00699,0.4,1.0,1.15,0.003215
357,2045-09-01,50970.449715,12902.292296,254.852249,12647.440047,121.078862,38201.930806,0.00699,0.4,1.0,1.13,0.003159
358,2045-10-01,38201.930806,12861.528408,191.009654,12670.518754,72.098543,25459.313509,0.00699,0.4,1.0,1.01,0.002824
359,2045-11-01,25459.313509,12825.208542,127.296568,12697.911975,30.685069,12730.716466,0.00699,0.4,1.0,0.86,0.002405
360,2045-12-01,12730.716466,12794.370048,63.653582,12730.716466,0.000000,0.000000,0.00699,0.4,1.0,0.90,0.002516


In [20]:
df[['Beg Bal','Payment','Interest', 'Principal', 'Prepayment','End Bal']] = \
    df[['Beg Bal','Payment','Interest', 'Principal', 'Prepayment','End Bal']].round(2)
df

,Payment Date,Beg Bal,Payment,Interest,Principal,Prepayment,End Bal,BaseFactor,Burnout,Seasoning,Seasonality,SMM
Period,,,,,,,,,,,,
1,2016-01-01,10000000.00,59955.05,50000.00,9955.05,1294.75,9988750.20,0.00699,1.0,0.020833,0.89,0.000130
2,2016-02-01,9988750.20,59947.28,49943.75,10003.53,2615.64,9976131.03,0.00699,1.0,0.041667,0.90,0.000262
3,2016-03-01,9976131.03,59931.57,49880.66,10050.91,4136.17,9961943.94,0.00699,1.0,0.062500,0.95,0.000415
4,2016-04-01,9961943.94,59906.70,49809.72,10096.98,5564.98,9946281.98,0.00699,1.0,0.083333,0.96,0.000559
5,2016-05-01,9946281.98,59873.20,49731.41,10141.79,7668.72,9928471.48,0.00699,1.0,0.104167,1.06,0.000772
...,...,...,...,...,...,...,...,...,...,...,...,...
356,2045-08-01,63759.98,12943.91,318.80,12625.11,164.42,50970.45,0.00699,0.4,1.000000,1.15,0.003215
357,2045-09-01,50970.45,12902.29,254.85,12647.44,121.08,38201.93,0.00699,0.4,1.000000,1.13,0.003159
358,2045-10-01,38201.93,12861.53,191.01,12670.52,72.10,25459.31,0.00699,0.4,1.000000,1.01,0.002824
